In [63]:
import requests
import json
import pandas as pd

#API call for json data of requested reports (in order: Civilian Labor Force, Civilian Employment, Civilian Employment, Unemployment Rate, Total Nonfarm Employment, Total Private Average Weekly Hours of All Employees, Total Private Average Weekly Hours of Prod. and Nonsup. Employees, Total Private Average Hourly Earnings of All Employees, Total Private Average Hourly Earnings of Prod. and Nonsup. Employees and Employment Cost Index (ECI) Civilian (Unadjusted))
headers = {'Content-type': 'application/json'}
data = json.dumps({"seriesid": ['LNS11000000', 'LNS12000000', 'LNS13000000', 'LNS14000000', 'CES0000000001', 'CES0500000007', 'CES0500000003', 'CES0500000008'],"startyear":"2023", "endyear":"2024"})
p = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
json_data = json.loads(p.text)
json_data = json_data['Results']['series']

#turn the returned json data dictionary into a pandas dataframe
json_data = pd.DataFrame(json_data)

#parse out the nested dictionary 'data' into individual data entries for each change in seriesID
separated_data = []
for index, row in json_data.iterrows():
    series_id = row['seriesID']
    data_list = row['data']

    if isinstance(data_list, list) and len(data_list) > 0 and isinstance(data_list[0], dict) :
        df = pd.DataFrame(data_list)
        df['seriesID'] = series_id
        separated_data.append(df)
    else:
        print(f"Warning: Unexpected 'data' format for seriesID: {series_id}")

final_df = pd.concat(separated_data, ignore_index=True)

#move column seriesID to the front of the dataframe
cols = list(final_df.columns)
cols.insert(0, cols.pop(cols.index('seriesID')))
final_df = final_df.loc[:, cols]

#removing uneeded columns that wont be used in the dashboard
final_df = final_df.drop(columns=['period', 'footnotes'])
final_df = final_df.rename(columns={'periodName': 'Month'})

#map series name to the corresponding serries number and create a new column for it
series_mapping = {
    "LNS11000000": "Civilian Labor Force",
    "LNS12000000": "Civilian Employment",
    "LNS13000000": "Civilian Unemployment",
    "LNS14000000": "Unemployment Rate",
    "CES0000000001": "Total Nonfarm Employment",
    "CES0500000002": "Total Private Average Weekly Hours of All Employees",
    "CES0500000007": "Total Private Average Weekly Hours of Prod. and Nonsup. Employees",
    "CES0500000003": "Total Private Average Hourly Earnings of All Employees",
    "CES0500000008": "Total Private Average Hourly Earnings of Prod. and Nonsup. Employees"
}
final_df['Series Name'] = final_df['seriesID'].map(series_mapping)

#move the new column Series Name inbetween columns seriesID and Year
cols = list(final_df.columns)
seriesID_index = cols.index('seriesID')
year_index = cols.index('year')
cols.remove('Series Name')
cols.insert(seriesID_index + 1, 'Series Name')
final_df = final_df[cols]

#write to csv
final_df.to_csv('Data.csv', index=False)